<a href="https://colab.research.google.com/github/lamtung1997/Solving_PDEs_Deep_learning/blob/main/3_Stokes_(inverse).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SETUP
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import time
import math
import itertools
import os
import logging
from numpy import loadtxt
import matplotlib.pyplot as plt
from keras.constraints import Constraint
import keras.backend as K

# number pi
pi = math.pi

# random_seed
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
# DATA
# load the dataset
with open('/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/3. Stokes (Inverse)/output_data (Neural).csv', 'r', encoding='utf-8-sig') as f: 
    data = np.genfromtxt(f, dtype=float, delimiter=',')

# convert numpy array, P_in and P_border to Tensorflow Dataset
xy = tf.convert_to_tensor(data[:, 0:2], dtype=tf.float32)
u_v_p = tf.convert_to_tensor(data[:, 2:5], dtype=tf.float32)

dat = tf.data.Dataset.from_tensor_slices((xy, u_v_p))

batch_size = 1
bs = int(len(data))

dat = dat.shuffle(buffer_size=bs).batch(batch_size).prefetch(128).cache()

In [ ]:
print("number of data: ", dat.cardinality().numpy()*batch_size)

number of data:  5625


In [ ]:
# Set condition for weight 'nu'
class Between(Constraint):
    def __init__(self, min_value, max_value):
        self.min_value = min_value
        self.max_value = max_value

    def __call__(self, nu):        
        return K.clip(nu, self.min_value, self.max_value)

    def get_config(self):
        return {'min_value': self.min_value, 'max_value': self.max_value}

In [ ]:
# MODEL
nn = 32
ini = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=1)

inputs = keras.Input(shape=(2,), name='points')
hidden_1 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_1')(inputs)
hidden_2 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_2')(hidden_1)
# hidden_3 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_3')(hidden_2)
# hidden_4 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_4')(hidden_3)
# hidden_5 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_5')(hidden_4)
# hidden_6 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_6')(hidden_5)
# hidden_7 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_7')(hidden_6)
# hidden_8 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_8')(hidden_7)
# hidden_9 = layers.Dense(nn, activation='tanh', kernel_initializer=ini, name='hidden_9')(hidden_8)
outputs = layers.Dense(3, activation='linear', kernel_initializer=ini, name="u_v_p")(hidden_2)
model = keras.Model(inputs=inputs, outputs=outputs, name='NV_inverse')
model.add_weight(name="nu", shape=(1,1), dtype=tf.float32, trainable=True, initializer=ini, constraint=Between(0, 100))
model.summary()

Model: "NV_inverse"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
points (InputLayer)          [(None, 2)]               0         
_________________________________________________________________
hidden_1 (Dense)             (None, 32)                96        
_________________________________________________________________
hidden_2 (Dense)             (None, 32)                1056      
_________________________________________________________________
u_v_p (Dense)                (None, 3)                 99        
Total params: 1,252
Trainable params: 1,252
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.weights[len(model.weights) - 1]

<tf.Variable 'nu:0' shape=(1, 1) dtype=float32, numpy=array([[0.00633252]], dtype=float32)>

In [ ]:
# CHOOSE OPTIMIZER
optimizer = tf.keras.optimizers.Adam()

In [ ]:
# TRAIN FUNCTION
@tf.function
def train_step(dat_batch):
    P = dat_batch[0]
    uvp = dat_batch[1]

    with tf.GradientTape(persistent=True) as tape:
        batch_loss = 0

        # loss
        for i in range(P.shape[0]):
            z = tf.reshape(P[i], shape=(1,2))
            with tf.GradientTape(persistent=True) as tape1:
                tape1.watch(z)
                with tf.GradientTape(persistent=True) as tape2:  
                    tape2.watch(z)
                    model_vals = model(z)

                    # calculating u_xx, u_yy, v_xx, v_yy, p_x, p_y
                    u = model_vals[0][0]
                    v = model_vals[0][1]
                    p = model_vals[0][2]

                u_x = tape2.gradient(u, z)[0][0]
                u_y = tape2.gradient(u, z)[0][1]
                v_x = tape2.gradient(v, z)[0][0]
                v_y = tape2.gradient(v, z)[0][1]
                p_x = tape2.gradient(p, z)[0][0]
                p_y = tape2.gradient(p, z)[0][1]
            
            u_xx = tape1.gradient(u_x, z)[0][0]
            u_yy = tape1.gradient(u_y, z)[0][1]
            v_xx = tape1.gradient(v_x, z)[0][0]
            v_yy = tape1.gradient(v_y, z)[0][1]

            # calculating loss
            nu = model.weights[len(model.weights) - 1]
            loss1 = tf.math.square(nu*(u_xx + u_yy) - p_x)
            loss2 = tf.math.square(nu*(v_xx + v_yy) - p_y)
            loss3 = tf.math.square(u_x + v_y)
            loss4 = tf.math.square(u - uvp[0][0]) + tf.math.square(v - uvp[0][1]) + tf.math.square(p - uvp[0][2])
            batch_loss += loss1 + loss2 + loss3 + loss4
    
    # update weights
    grads = tape.gradient(batch_loss, model.trainable_weights)          # gradient of loss function with respect to w
    optimizer.apply_gradients(zip(grads, model.trainable_weights))      # update w

    return batch_loss

In [ ]:
# TRAIN
losses = []
def train(learning_rate):
    optimizer.learning_rate.assign(learning_rate)
    loss = 0
    for dat_batch in dat:
        batch_loss = train_step(dat_batch)
        loss += batch_loss
    losses.append(loss.numpy())

In [15]:
i = 0
max_ite = 1000
lr = 1e-3
start = time.time()
print("Learning rate:", lr)
while(True):
    start_ite = time.time()
    train(lr)
    if(i%1 == 0):
        print("[%3s] loss = %13.7f \t %5.3fs \t less than %5.3f minutes to finish \t nu: %5.4f" % (i, losses[i], time.time() - start_ite, (time.time() - start_ite)*(max_ite - i)/60, model.weights[len(model.weights) - 1]))
    if(i > 200):
        if(losses[i] / np.min(losses[i-200: i - 100]) > 0.9):
            lr = lr/10
            if(lr < 1e-5):
                break
            print("\nNew learning rate:", lr)
    if(i > max_ite):
        break
    i = i + 1

print("Total time: %.2f minutes" % ((time.time() - start)/60))

Learning rate: 0.001
[  0] loss = 24358.7285156 	 8.788s 	 less than 146.473 minutes to finish 	 nu: 3.0232
[  1] loss = 10508.4238281 	 6.409s 	 less than 106.710 minutes to finish 	 nu: 3.4672
[  2] loss =  3450.4973145 	 6.390s 	 less than 106.291 minutes to finish 	 nu: 3.4812
[  3] loss =  1787.2462158 	 6.351s 	 less than 105.538 minutes to finish 	 nu: 3.4528
[  4] loss =  1520.6289062 	 6.269s 	 less than 104.058 minutes to finish 	 nu: 3.4249
[  5] loss =  1404.0858154 	 6.384s 	 less than 105.868 minutes to finish 	 nu: 3.4067
[  6] loss =  1336.6989746 	 6.510s 	 less than 107.842 minutes to finish 	 nu: 3.3938
[  7] loss =  1281.3936768 	 6.407s 	 less than 106.036 minutes to finish 	 nu: 3.3853
[  8] loss =  1235.4283447 	 6.398s 	 less than 105.785 minutes to finish 	 nu: 3.3804
[  9] loss =  1196.5352783 	 6.607s 	 less than 109.128 minutes to finish 	 nu: 3.3781
[ 10] loss =  1162.2271729 	 6.269s 	 less than 103.447 minutes to finish 	 nu: 3.3774
[ 11] loss =  1130.511

In [16]:
model.save("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/3. Stokes (Inverse)")

INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/3. Stokes (Inverse)/assets


In [17]:
model1 = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/3. Stokes (Inverse)")